## Evaluate Multiple Classifiers Runtime

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, concatenate_datasets
from sklearn.metrics import accuracy_score
import random
from tqdm import tqdm
import numpy as np
import torch
import os

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set Random Seed for Reproducibility

In [4]:
# Set a seed for random module
random.seed(42)

# Set a seed for numpy module
np.random.seed(42)

# Set a seed for torch module
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Define Parameters

In [5]:
SAMPLING = "random" # "random", "stratified", "clustered", "shared_domain"
SUFFIX = "_extended" #"", "_holdout", "_extended"
SPLIT = "test" # "train", "test", "holdout", "extende
MAX_CONTENT_LENGTH = 384 # 496, 192
OVERLAP = 64
FEATURES = "url_and_content" # "url", "content", "url_and_content"
FOLDER_DATA = "data"
FOLDER_MODELS = "models"
N = 5  # Number of times to run the evaluation

In [6]:
TOPICS = ["cannabis", "kinder", "energie"]
#TOPICS = ["cannabis"]

In [7]:
MODELS = ["distilbert/distilbert-base-multilingual-cased",
          "google-bert/bert-base-multilingual-cased", 
          "FacebookAI/xlm-roberta-base", 
          "FacebookAI/xlm-roberta-large", 
          "dbmdz/bert-base-german-uncased", 
          "deepset/gelectra-large",
          "deepset/gelectra-base",
          "deepset/gbert-large",
          "deepset/gbert-base",
          ]

In [8]:
CUDA_ID = 1

**Extract URL-path:**

In [9]:
from urllib.parse import urlparse, urlunparse

def extract_url_path(example):
    view_url = example['view_url']
    if "://" not in view_url:
        view_url = "http://" + view_url  # Assume http if no protocol specified
    parsed_url = urlparse(view_url)
    new_url = urlunparse(('', '', parsed_url.path, parsed_url.params, parsed_url.query, parsed_url.fragment))
    example['url_path'] = new_url.lstrip('/')  # Store the result in a new field
    return example


extract_url_path({"view_url": "https://www.google.com/search?q=python+url+path"})

{'view_url': 'https://www.google.com/search?q=python+url+path',
 'url_path': 'search?q=python+url+path'}

## Evaluate Models

In [10]:
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
import time
import numpy as np

class TokenizedDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

def get_predictions(tokenized_datasets, tokenizer, model, device, features, split="test", batch_size=32):
    """Use the trained model to make predictions on the test set."""
    
    # Tokenize the dataset
    texts = []
    urls = []
    labels = []
    
    for row in tokenized_datasets[split]:
        labels.append(row["label"])
        if features == "content":
            texts.append(row["text"])
        elif features == "url":
            urls.append(row["url_path"])
        elif features == "url_and_content":
            urls.append(row["url_path"])
            texts.append(row["text"])
        else:
            raise ValueError("Invalid value for FEATURES. Expected 'content', 'url', or 'url_and_content'.")

    if features == "content":
        encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    elif features == "url":
        encodings = tokenizer(urls, padding=True, truncation=True, return_tensors="pt")
    elif features == "url_and_content":
        encodings = tokenizer(urls, texts, padding=True, truncation=True, return_tensors="pt")
    
    dataset = TokenizedDataset(encodings, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    preds = []
    probabilities = []
    inference_times = []
    
    model.eval()  # Put the model in evaluation mode

    for batch in tqdm(dataloader):
        # Move inputs to device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        start_time = time.time()
        with torch.no_grad():
            # Forward pass
            outputs = model(**batch)
            # Apply softmax to logits to get probabilities
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            # Get the predicted class (the one with the highest probability)
            predicted_classes = torch.argmax(predictions, dim=-1).cpu().numpy()
        
        end_time = time.time()
        batch_inference_time = end_time - start_time
        inference_times.extend([batch_inference_time / len(batch["labels"])] * len(batch["labels"]))  # Store time per sample
        
        # Store the predictions and probabilities
        preds.extend(predicted_classes)
        probabilities.extend(predictions[:, 1].cpu().numpy().tolist())  # Store the probability of the positive class
    
    # Calculate statistics
    avg_time = np.mean(inference_times)
    std_time = np.std(inference_times)
    min_time = np.min(inference_times)
    max_time = np.max(inference_times)
    
    return preds, labels, probabilities, avg_time, std_time, min_time, max_time, inference_times


In [11]:
def calc_metrics(labels, preds):
    """
    Calculates the accuracy, precision, recall, and F1 score for the given labels and predictions and returns them in a dictionary.
    """
    
    metrics = {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='binary'),
        'recall': recall_score(labels, preds, average='binary'),
        'f1': f1_score(labels, preds, average='binary'),
    }

    return metrics

In [12]:
def sample_random_from_dataset(dataset, n=5, subset='test'):
    """
    Samples n random examples from a specified subset of the dataset.
    """
    n = min(n, len(dataset[subset]))
    random_indices = random.sample(range(len(dataset[subset])), n)
    sampled_dataset = dataset[subset].select(random_indices)
    return sampled_dataset

**Get chunk level predictions:**

In [13]:
from collections import defaultdict
eval_results = defaultdict(lambda: defaultdict(list))

In [11]:
for topic in TOPICS: # ----------------------------------------------------------------------

    for model_name in MODELS: # -------------------------------------------------------------

        print(f"\n\n###### Evaluating model {model_name} on {topic} ###### \n\n")
            
        if FEATURES == "url":
            dataset = load_from_disk(
                f"../../{FOLDER_DATA}/tmp/processed_dataset_{topic}_buffed_{SAMPLING}{SUFFIX}")

            if SPLIT == "holdout":
                dataset["holdout"] = concatenate_datasets(
                    [dataset["holdout"], dataset["test"]])
            # Extract the path from the URL
            dataset = dataset.map(extract_url_path, num_proc=8)
        else:
            dataset = load_from_disk(
                f"../../{FOLDER_DATA}/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}")

            if SPLIT == "holdout":
                dataset["holdout"] = concatenate_datasets(
                    [dataset["holdout"], dataset["test"]])
                
            # Extract the path from the URL
            dataset = dataset.map(extract_url_path)
        
        for i in range(N):
            print(f"Run {i+1}/{N}")
            
            # Load model and tokenizer
            model_name_local = f"../../{FOLDER_MODELS}/{model_name.replace('/','_')}_{topic}_model_{FEATURES}/"
            print(f"Loading model from {model_name_local}")
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name_local, num_labels=2, local_files_only=True)

            # Move model to GPU if available
            DEVICE = torch.device(f"cuda:{CUDA_ID}" if torch.cuda.is_available() else "cpu")
            model.to(DEVICE)
            
            # Use the trained model to make predictions on the test set
            preds, labels, probas, avg_time, std_time, min_time, max_time, runtimes = get_predictions(dataset, tokenizer, model, DEVICE, FEATURES, split=SPLIT)
            metrics = calc_metrics(labels, preds)
            
            print(f"Metrics for {model_name} on {topic}: {metrics}")
            print(f'Average Inference Time per Sample: {avg_time:.6f} seconds')
            print(f'STD Inference Time per Sample: {std_time:.6f} seconds')
            print(f'Min Inference Time per Sample: {min_time:.6f} seconds')
            print(f'Max Inference Time per Sample: {max_time:.6f} seconds')
            
            # Update the eval_results dictionary
            eval_results[model_name][topic].append({
                'metrics': metrics,
                'avg_time': avg_time,
                'std_time': std_time,
                'min_time': min_time,
                'max_time': max_time
            })
            
            # Clear GPU memory to avoid memory errors
            del model, tokenizer
            torch.cuda.empty_cache()



###### Evaluating model distilbert/distilbert-base-multilingual-cased on cannabis ###### 


Run 1/5
Loading model from ../../models/distilbert_distilbert-base-multilingual-cased_cannabis_model_url_and_content/


100%|██████████| 16/16 [00:04<00:00,  3.72it/s]


Metrics for distilbert/distilbert-base-multilingual-cased on cannabis: {'accuracy': 0.9960552268244576, 'precision': 0.9939759036144579, 'recall': 1.0, 'f1': 0.9969788519637462}
Average Inference Time per Sample: 0.008373 seconds
STD Inference Time per Sample: 0.002354 seconds
Min Inference Time per Sample: 0.007752 seconds
Max Inference Time per Sample: 0.017443 seconds
Run 2/5
Loading model from ../../models/distilbert_distilbert-base-multilingual-cased_cannabis_model_url_and_content/


100%|██████████| 16/16 [00:03<00:00,  4.01it/s]


Metrics for distilbert/distilbert-base-multilingual-cased on cannabis: {'accuracy': 0.9960552268244576, 'precision': 0.9939759036144579, 'recall': 1.0, 'f1': 0.9969788519637462}
Average Inference Time per Sample: 0.007763 seconds
STD Inference Time per Sample: 0.000004 seconds
Min Inference Time per Sample: 0.007756 seconds
Max Inference Time per Sample: 0.007774 seconds
Run 3/5
Loading model from ../../models/distilbert_distilbert-base-multilingual-cased_cannabis_model_url_and_content/


...

Metrics for deepset/gbert-base on energie: {'accuracy': 0.998272884283247, 'precision': 0.9962264150943396, 'recall': 1.0, 'f1': 0.998109640831758}
Average Inference Time per Sample: 0.013726 seconds
STD Inference Time per Sample: 0.000113 seconds
Min Inference Time per Sample: 0.013648 seconds
Max Inference Time per Sample: 0.015219 seconds
Run 4/5
Loading model from ../../models/deepset_gbert-base_energie_model_url_and_content/


100%|██████████| 19/19 [00:07<00:00,  2.38it/s]


Metrics for deepset/gbert-base on energie: {'accuracy': 0.998272884283247, 'precision': 0.9962264150943396, 'recall': 1.0, 'f1': 0.998109640831758}
Average Inference Time per Sample: 0.013714 seconds
STD Inference Time per Sample: 0.000116 seconds
Min Inference Time per Sample: 0.013654 seconds
Max Inference Time per Sample: 0.015278 seconds
Run 5/5
Loading model from ../../models/deepset_gbert-base_energie_model_url_and_content/


100%|██████████| 19/19 [00:08<00:00,  2.37it/s]

Metrics for deepset/gbert-base on energie: {'accuracy': 0.998272884283247, 'precision': 0.9962264150943396, 'recall': 1.0, 'f1': 0.998109640831758}
Average Inference Time per Sample: 0.013711 seconds
STD Inference Time per Sample: 0.000104 seconds
Min Inference Time per Sample: 0.013627 seconds
Max Inference Time per Sample: 0.015078 seconds


In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

german_stop_words = stopwords.words('german')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jschelb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import numpy as np
import time
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


for topic in TOPICS:
    print(f"\n\n###### Evaluating SVM on {topic} ###### \n\n")
    
    if FEATURES == "url":
        dataset = load_from_disk(
            f"../../{FOLDER_DATA}/tmp/processed_dataset_{topic}_buffed_{SAMPLING}{SUFFIX}")

        if SPLIT == "holdout":
            dataset["holdout"] = concatenate_datasets(
                [dataset["holdout"], dataset["test"]])
        # Extract the path from the URL
        dataset = dataset.map(extract_url_path, num_proc=8)
    else:
        dataset = load_from_disk(
            f"../../{FOLDER_DATA}/tmp/processed_dataset_{topic}_buffed_chunkified_{SAMPLING}{SUFFIX}_{MAX_CONTENT_LENGTH}")

        if SPLIT == "holdout":
            dataset["holdout"] = concatenate_datasets(
                [dataset["holdout"], dataset["test"]])
             
        # Extract the path from the URL
        dataset = dataset.map(extract_url_path)
        
        print(f"Loading dataset for {topic} from {dataset}")
        
        train_data = dataset['train']
        test_data = dataset['test']

        vectorizer = TfidfVectorizer(stop_words=german_stop_words, max_features=10000)

        # Vectorize the training data
        X_train = vectorizer.fit_transform(train_data['text'])

        # Vectorize the test data using the same vectorizer
        X_test = vectorizer.transform(test_data['text'])

        y_train = np.array(train_data['label'])
        y_test = np.array(test_data['label'])

        # Initialize and train the SVM classifier
        svm_classifier = SVC()

        # Train the classifier
        svm_classifier.fit(X_train, y_train)

    
    for i in range(N):
       for i in range(N):
        print(f"Run {i+1}/{N}")
    
        # Measure the total time for predicting the entire test set
        start_time = time.time()
        predictions = svm_classifier.predict(X_test)
        end_time = time.time()

        # Calculate average time per sample
        total_time = end_time - start_time
        num_samples = X_test.shape[0]
        avg_time_per_sample = total_time / num_samples
        runtimes = [avg_time_per_sample] * num_samples
        std_time = np.std(runtimes)
        min_time = np.min(runtimes)
        max_time = np.max(runtimes)
        
        # Collecting metrics
        metrics = classification_report(y_test, predictions, output_dict=True)

        print(f"Metrics for SVM on {topic}: {metrics}")
        print(f'Average Inference Time per Sample: {avg_time_per_sample:.6f} seconds')
        print(f'STD Inference Time per Sample: {std_time:.6f} seconds')
        print(f'Min Inference Time per Sample: {min_time:.6f} seconds')
        print(f'Max Inference Time per Sample: {max_time:.6f} seconds')

        # Update eval_results
        eval_results['svm_classifier'][topic].append({
            'metrics': metrics,
            'avg_time': avg_time_per_sample,
            'std_time': std_time,
            'min_time': min_time,
            'max_time': max_time
        })



###### Evaluating SVM on cannabis ###### 


Loading dataset for cannabis from DatasetDict({
    train: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'chunk_id', 'url_path'],
        num_rows: 3815
    })
    test: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'chunk_id', 'url_path'],
        num_rows: 507
    })
    holdout: Dataset({
        features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'token_count', 'chunk_id', 'url_path'],
        num_rows: 33702
   

In [27]:
print(eval_results)

defaultdict(<function <lambda> at 0x7fb1d350f370>, {'svm_classifier': defaultdict(<class 'list'>, {'cannabis': [{'metrics': {'0': {'precision': 0.7953488372093023, 'recall': 0.9661016949152542, 'f1-score': 0.8724489795918368, 'support': 177.0}, '1': {'precision': 0.9794520547945206, 'recall': 0.8666666666666667, 'f1-score': 0.9196141479099679, 'support': 330.0}, 'accuracy': 0.9013806706114399, 'macro avg': {'precision': 0.8874004460019114, 'recall': 0.9163841807909605, 'f1-score': 0.8960315637509023, 'support': 507.0}, 'weighted avg': {'precision': 0.9151793338624028, 'recall': 0.9013806706114399, 'f1-score': 0.9031482015740523, 'support': 507.0}}, 'avg_time': 0.0017493927972556571, 'std_time': 0.0004228871436871009, 'min_time': 0.0008242130279541016, 'max_time': 0.008214235305786133}, {'metrics': {'0': {'precision': 0.7953488372093023, 'recall': 0.9661016949152542, 'f1-score': 0.8724489795918368, 'support': 177.0}, '1': {'precision': 0.9794520547945206, 'recall': 0.8666666666666667, '

### Save Chunk Level Predictions and Output Results

In [20]:
from IPython.display import display, HTML
from tabulate import tabulate
import json

In [14]:
# Define the file path to save the dictionary
file_path =f"eval_runtime_{FEATURES}_{SPLIT}_chunks.json"

In [21]:
# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(eval_results, file)

NameError: name 'file_path' is not defined

In [16]:
with open(file_path, "r") as file:
    eval_results = json.load(file)

In [30]:
from collections import defaultdict
from tabulate import tabulate
import numpy as np

# Function to compute average metrics and runtime statistics
def compute_average_metrics(eval_results):
    averaged_results = defaultdict(dict)
    
    for model, topics_metrics in eval_results.items():
        for topic, runs in topics_metrics.items():
            avg_metrics = defaultdict(list)
            for run in runs:
                metrics = run.get('metrics', {})
                for key, value in metrics.items():
                    if isinstance(value, (int, float)):  # Ensure the value is numeric
                        avg_metrics[key].append(value)
                avg_metrics['avg_time'].append(run.get('avg_time', 0))
                avg_metrics['std_time'].append(run.get('std_time', 0))
                avg_metrics['min_time'].append(run.get('min_time', 0))
                avg_metrics['max_time'].append(run.get('max_time', 0))
            
            # Calculate means only if there are values to avoid empty list issues
            averaged_results[model][topic] = {metric: np.mean(values) for metric, values in avg_metrics.items() if len(values) > 0 and isinstance(values[0], (int, float))}
            averaged_results[model][topic].update({
                'avg_time_std': np.std(avg_metrics['avg_time']) if len(avg_metrics['avg_time']) > 0 else 0,
                'std_time_std': np.std(avg_metrics['std_time']) if len(avg_metrics['std_time']) > 0 else 0,
                'min_time_std': np.std(avg_metrics['min_time']) if len(avg_metrics['min_time']) > 0 else 0,
                'max_time_std': np.std(avg_metrics['max_time']) if len(avg_metrics['max_time']) > 0 else 0,
            })
    
    return averaged_results

# Compute average metrics
averaged_eval_results = compute_average_metrics(eval_results)

# Identify all topics (assuming all models are evaluated on the same topics)
topics = list(next(iter(averaged_eval_results.values())).keys())

# Prepare headers for the table: each topic will have six metrics
headers = ["Model"] + [f"{topic} {metric}" for topic in topics for metric in ["Acc.", "Prec.", "Rec.", "F1", "Avg. Time", "STD Time"]]

# Prepare rows: one row per model, containing metrics for each topic
rows = []
for model, topics_metrics in averaged_eval_results.items():
    row = [model]  # Start with the model name
    for topic in topics:
        metrics = topics_metrics.get(topic, {})
        row.extend([
            metrics.get('accuracy', 0.0),
            metrics.get('precision', 0.0),
            metrics.get('recall', 0.0),
            metrics.get('f1-score', 0.0),  # Assuming the key for F1 score is 'f1-score'
            metrics.get('avg_time', 0.0),
            metrics.get('std_time', 0.0)
        ])
    rows.append(row)

# Generate the HTML table
table_html = tabulate(rows, headers=headers, tablefmt="html", showindex="never", floatfmt=".3f")

print(table_html)  # This will print the HTML table; in a real application, you might save this to an HTML file.


<table>
<thead>
<tr><th>Model         </th><th style="text-align: right;">  cannabis Acc.</th><th style="text-align: right;">  cannabis Prec.</th><th style="text-align: right;">  cannabis Rec.</th><th style="text-align: right;">  cannabis F1</th><th style="text-align: right;">  cannabis Avg. Time</th><th style="text-align: right;">  cannabis STD Time</th><th style="text-align: right;">  kinder Acc.</th><th style="text-align: right;">  kinder Prec.</th><th style="text-align: right;">  kinder Rec.</th><th style="text-align: right;">  kinder F1</th><th style="text-align: right;">  kinder Avg. Time</th><th style="text-align: right;">  kinder STD Time</th><th style="text-align: right;">  energie Acc.</th><th style="text-align: right;">  energie Prec.</th><th style="text-align: right;">  energie Rec.</th><th style="text-align: right;">  energie F1</th><th style="text-align: right;">  energie Avg. Time</th><th style="text-align: right;">  energie STD Time</th></tr>
</thead>
<tbody>
<tr><td>sv

In [29]:
from IPython.display import display, HTML
display(HTML(table_html))


Model,cannabis Acc.,cannabis Prec.,cannabis Rec.,cannabis F1,cannabis Avg. Time,cannabis STD Time,kinder Acc.,kinder Prec.,kinder Rec.,kinder F1,kinder Avg. Time,kinder STD Time,energie Acc.,energie Prec.,energie Rec.,energie F1,energie Avg. Time,energie STD Time
svm_classifier,0.901,0.000,0.000,0.000,0.001,0.000,0.953,0.000,0.000,0.000,0.001,0.000,0.945,0.000,0.000,0.000,0.001,0.000


In [19]:
# Compute average metrics
averaged_eval_results = compute_average_metrics(eval_results)

# Prepare headers for the table
headers = ["Model", "Avg. F1 Score", "Avg. Runtime", "Runtime STD"]

# Prepare rows: one row per model, containing average F1 score and average runtime
rows = []
for model, topics_metrics in averaged_eval_results.items():
    avg_f1_scores = []
    avg_times = []
    for topic, metrics in topics_metrics.items():
        avg_f1_scores.append(metrics.get('f1', 0.0))
        avg_times.append(metrics.get('avg_time', 0.0))
    
    avg_f1_score = np.mean(avg_f1_scores)
    avg_runtime = np.mean(avg_times)
    runtime_std = np.std(avg_times)
    
    row = [model, avg_f1_score, avg_runtime, runtime_std]
    rows.append(row)

# Generate the HTML table
table_html = tabulate(rows, headers=headers, tablefmt="html", showindex="never", floatfmt=".3f")

In [20]:
from IPython.display import display, HTML
display(HTML(table_html))

Model,Avg. F1 Score,Avg. Runtime,Runtime STD
distilbert/distilbert-base-multilingual-cased,0.997,0.008,0.000
google-bert/bert-base-multilingual-cased,0.999,0.017,0.001
FacebookAI/xlm-roberta-base,0.994,0.016,0.001
FacebookAI/xlm-roberta-large,0.997,0.051,0.002
dbmdz/bert-base-german-uncased,0.999,0.015,0.001
deepset/gelectra-large,0.998,0.052,0.005
deepset/gelectra-base,0.993,0.016,0.002
deepset/gbert-large,0.995,0.052,0.005
deepset/gbert-base,0.998,0.016,0.002
